# Lecture, nettoyage & export des données depuis un fichier csv
## En 3 Parties:
1 - Lecture du fichier et transformation en dataframe
2 - Nettoyage des données
3 - Export des données

In [26]:
import pandas as pd
import os
import numpy as np
from pathlib import Path

CSV_FILE_PATH = Path('../../../data_src/produits.csv')  

# 1 - Lecture du fichier et transformation en dataframe
products_df = pd.read_csv(CSV_FILE_PATH)
display(products_df.head())


,nom_produit,code_produit,prix
0,Clavier mécanique,pr1,"89,99€"
1,souris optique,P r-02,14.5 €
2,Écran 27 Pouces,PROD3,279.0 euros
3,CHAise de Bureau,pR-004,$45.99
4,Bureau d'angle,PRODUIT5,230 €


In [ ]:

# 2 - Nettoyage des données
# Suppression des doublons
products_df.drop_duplicates(inplace=True)
EUR_USD_RATE = 0.86  # taux de conversion dollar vers euro (28/10/2025)

# Normaliser et convertir la colonne 'prix' en float, passage dollar vers euro si besoin
# Passer tous les prix en string & détecter les prix en dollars
raw_price = products_df["prix"].astype(str)
dollars_mask = raw_price.str.contains(r"\$", regex=True, na=False)

# Repasser en float après identification des prix en dollars
cleaned = (
    raw_price
    .str.replace(r"[^\d,.\-]", "", regex=True)
    .str.replace(",", ".", regex=False)
    .replace("", np.nan)
)

products_df["prix"] = pd.to_numeric(cleaned, errors="coerce")

# Conversion des prix en dollars vers euros
products_df.loc[dollars_mask, "prix"] = products_df.loc[dollars_mask, "prix"] * EUR_USD_RATE
display(products_df.head())



,nom_produit,code_produit,prix
0,clavier mecanique,PR-001,89.99
1,souris optique,PR-002,14.50
2,ecran 27 pouces,PR-003,279.00
3,chaise de bureau,PR-004,45.99
4,bureau d'angle,PR-005,230.00


In [28]:
# Normaliser la colonne code_produit
# Format cible PR-XXX

products_df["code_produit"] = products_df["code_produit"].astype(str).str.upper()

# extraire le dernier groupe de chiffres et formater en PR-XXX (XXX zéro-paddé 3 chiffres)
nums = products_df["code_produit"].str.extract(r'(\d+)(?!.*\d)', expand=False)

products_df["code_produit"] = np.where(
    nums.notna(),
    'PR-' + nums.astype(int).astype(str).str.zfill(3),
    np.nan
)

display(products_df.head())

,nom_produit,code_produit,prix
0,Clavier mécanique,PR-001,89.99
1,souris optique,PR-002,14.50
2,Écran 27 Pouces,PR-003,279.00
3,CHAise de Bureau,PR-004,45.99
4,Bureau d'angle,PR-005,230.00


In [29]:
# Normaliser la colonne nom_produit
# Format cible: lower case, sans espaces superflus
products_df["nom_produit"] = (
    products_df["nom_produit"]
    .astype(str)
    .str.strip()
    .str.normalize('NFKD')
    .str.encode('ascii', errors='ignore').str.decode('utf-8')  # Supprime les accents
    .str.lower()  # lower case
    .replace("Nan", np.nan)  # Convertit 'Nan' en NaN
)
display(products_df.head())

,nom_produit,code_produit,prix
0,clavier mecanique,PR-001,89.99
1,souris optique,PR-002,14.50
2,ecran 27 pouces,PR-003,279.00
3,chaise de bureau,PR-004,45.99
4,bureau d'angle,PR-005,230.00


In [30]:
# 3 - Export des données
OUTPUT_CSV_FILE_PATH = Path('../../../data_cleaned/produits_cleaned.csv')
products_df.to_csv(OUTPUT_CSV_FILE_PATH, index=False)